In [1]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np, seaborn as sns, sklearn as sk
import csv

### Tournmanet Dataframe

In [2]:
# Import tournaments data and create dataframe, adding column headings.
tournaments = pd.read_csv('data/tournaments_1877-2017_UNINDEXED.csv', header=None)
tournamentscolumns = ['tourney_year','tourney_order','tourney_name','tourney_id','tourney_slug','tourney_location',
                      'tourney_dates','tourney_month','tourney_day','tourney_singles_draw','tourney_doubles_draw',
                      'tourney_conditions','tourney_surface','tourney_fin_commit','tourney_url_suffix','singles_winner_name',
                      'singles_winner_url','singles_winner_player_slug','singles_winner_player_id','doubles_winner_1_name',
                      'doubles_winner_1_url','doubles_winner_1_player_slug','doubles_winner_1_player_id','doubles_winner_2_name',
                      'doubles_winner_2_url','doubles_winner_2_player_slug','doubles_winner_2_player_id','tourney_year_id']
tournaments.columns = tournamentscolumns

In [3]:
# Drop unnecessary columns in tournaments. (Only keep 'tourney_dates' and 'tourney_year_id'). Will allow match dates to be included in model.
tournaments.drop(["tourney_year","tourney_order","tourney_name","tourney_id","tourney_slug","tourney_location",
                  "tourney_month","tourney_day","tourney_singles_draw","tourney_doubles_draw","tourney_conditions","tourney_surface",
                  "tourney_fin_commit","tourney_url_suffix","singles_winner_name","singles_winner_url","singles_winner_player_slug",
                  "singles_winner_player_id","doubles_winner_1_name","doubles_winner_1_url","doubles_winner_1_player_slug",
                  "doubles_winner_1_player_id","doubles_winner_2_name","doubles_winner_2_url","doubles_winner_2_player_slug",
                  "doubles_winner_2_player_id"], inplace=True, axis=1)

In [4]:
# Convert tournament date strings to datetime objects.
tournaments['tourney_dates'] = pd.to_datetime(pd.Series(tournaments['tourney_dates']))

In [5]:
tournaments = tournaments[(tournaments['tourney_dates'].dt.year > 1989) ]

### Scores Dataframe

In [6]:
# Import scores data and create dataframe, adding column headings.
scores = pd.read_csv('data/match_scores_1991-2016_unindexed.csv', header=None)
scorescolumns = ['tourney_year_id','tourney_order','tourney_slug','tourney_url_suffix','tourney_round_name','round_order',
                 'match_order','winner_name','winner_player_id','winner_slug','loser_name','loser_player_id','loser_slug',
                 'winner_seed','loser_seed','match_score_tiebreaks','winner_sets_won','loser_sets_won','winner_games_won',
                 'loser_games_won','winner_tiebreaks_won','loser_tiebreaks_won','match_id','match_stats_url_suffix']
scores.columns = scorescolumns

In [7]:
# Drop unnecessary columns in scores.
scores.drop(["tourney_order","tourney_slug","tourney_url_suffix","tourney_round_name","round_order","match_order","winner_name",
             "winner_slug","loser_name","loser_slug","winner_seed","loser_seed","match_stats_url_suffix"], inplace=True, axis=1)

In [8]:
# Drop empty rows in scores.
nan_rows_scores = scores[ (scores['match_score_tiebreaks'].isnull() == True) ].index
scores.drop(nan_rows_scores, inplace=True)

### Winner and Loser Dataframes

In [9]:
winner = pd.DataFrame([scores.tourney_year_id, scores.winner_player_id, scores.match_score_tiebreaks,
                       scores.winner_sets_won, scores.winner_games_won, scores.winner_tiebreaks_won, scores.match_id]).transpose()

In [10]:
loser = pd.DataFrame([scores.tourney_year_id, scores.loser_player_id, scores.match_score_tiebreaks,
                       scores.loser_sets_won, scores.loser_games_won, scores.loser_tiebreaks_won, scores.match_id]).transpose()

In [11]:
winner = winner.rename(columns={'winner_player_id': 'player_id'})
loser = loser.rename(columns={'loser_player_id': 'player_id'})

In [12]:
winner = pd.merge(left=winner,right=tournaments,how='left',left_on='tourney_year_id',right_on='tourney_year_id')

In [13]:
loser = pd.merge(left=loser,right=tournaments,how='left',left_on='tourney_year_id',right_on='tourney_year_id')

### Rankings Dataframe

In [14]:
rankings = pd.read_csv('data/rankings_1973-2017_csv.csv', header=0, low_memory=False)

In [15]:
# Drop rows before 1990.
drop_rank_rows = rankings[ (rankings['week_year'] < 1990)].index
rankings = rankings.drop(drop_rank_rows)

In [16]:
# Drop unnecessary columns in rankings.
rankings.drop(["move_positions","move_direction","player_age","tourneys_played","player_url","player_slug"], inplace=True, axis=1)

In [17]:
# Convert rankings date strings to datetime objects.
rankings['week_title'] = pd.to_datetime(pd.Series(rankings['week_title']))

In [18]:
# Create 'date' column for later merging.
rankings['date'] = rankings['week_title']

In [19]:
# sort rankings by year, week, day and rank.
rankings = rankings.sort_values(['week_title'])

### Create dataframe to round dates.

In [20]:
# Create dataframe of all dates from Jan 1, 1991 to Dec 31, 2017.
df=pd.DataFrame({'date':pd.date_range('1990-12-15','2017-12-31')})

In [21]:
# Create reference column for the beginning day of the week for each day.
df['BeginWeek']=np.where(df.date.dt.weekday==0, # offset on Non Mondays only
                        df['date'],
                        df['date']-pd.DateOffset(weekday=0,weeks=1),
                        )

C:\Users\ironm\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:837: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  PerformanceWarning,


### Add rounding dates to model dataframes for merging.

In [22]:
rankings = pd.merge(left=rankings,right=df,how='left',left_on='date',right_on='date')

In [23]:
winner = pd.merge(left=winner,right=df,how='left',left_on='tourney_dates', right_on='date')

In [24]:
loser = pd.merge(left=loser,right=df,how='left',left_on='tourney_dates', right_on='date')

### Clean rankings/winner/loser of redundant date columns.

In [25]:
# Drop unnecessary columns in rankings/winner/loser.
rankings.drop(["week_title","date","week_year","week_month","week_day"], inplace=True, axis=1)
winner.drop(["tourney_dates","date"], inplace=True, axis=1)
loser.drop(["tourney_dates","date"], inplace=True, axis=1)

### Merge rankings into winner and loser dataframes.

In [26]:
winner1 = pd.merge(winner,rankings,on=["player_id","BeginWeek"], how="left")

In [27]:
loser1 = pd.merge(loser,rankings,on=["player_id","BeginWeek"], how="left")

In [28]:
# Label specific columns by dataframe they come from in the respective columns to identify winner and loser info.
winner1 = winner1.rename(columns={'player_id': 'winner_player_id'})
winner1 = winner1.rename(columns={'rank_text': 'winner_rank_text'})
winner1 = winner1.rename(columns={'rank_number': 'winner_rank_number'})
winner1 = winner1.rename(columns={'ranking_points': 'winner_ranking_points'})
loser1 = loser1.rename(columns={'player_id': 'loser_player_id'})
loser1 = loser1.rename(columns={'rank_text': 'loser_rank_text'})
loser1 = loser1.rename(columns={'rank_number': 'loser_rank_number'})
loser1 = loser1.rename(columns={'ranking_points': 'loser_ranking_points'})

In [29]:
# Drop empty rows in winner1/loser1 where rankings are missing.
nan_rows_winner1 = winner1[ (winner1['winner_rank_text'].isnull() == True) ].index
winner1.drop(nan_rows_winner1, inplace=True)

nan_rows_loser1 = loser1[ (loser1['loser_rank_text'].isnull() == True) ].index
loser1.drop(nan_rows_loser1, inplace=True)

In [30]:
winner1.isnull().sum()
loser1.isnull().sum()

tourney_year_id          0
loser_player_id          0
match_score_tiebreaks    0
loser_sets_won           0
loser_games_won          0
loser_tiebreaks_won      0
match_id                 0
BeginWeek                0
loser_rank_text          0
loser_rank_number        0
loser_ranking_points     0
dtype: int64

### Match Statistics Dataframe

In [31]:
matchstats = pd.read_csv('data/match_stats_1991-2016_unindexed.csv', header=None)
matchstatscolumns = ['tourney_order','match_id','match_stats_url_suffix','match_time','match_duration','winner_aces',
                     'winner_double_faults','winner_first_serves_in','winner_first_serves_total','winner_first_serve_points_won',
                     'winner_first_serve_points_total','winner_second_serve_points_won','winner_second_serve_points_total',
                     'winner_break_points_saved','winner_break_points_serve_total','winner_service_points_won',
                     'winner_service_points_total','winner_first_serve_return_won','winner_first_serve_return_total',
                     'winner_second_serve_return_won','winner_second_serve_return_total','winner_break_points_converted',
                     'winner_break_points_return_total','winner_service_games_played','winner_return_games_played',
                     'winner_return_points_won','winner_return_points_total','winner_total_points_won','winner_total_points_total',
                     'loser_aces','loser_double_faults','loser_first_serves_in','loser_first_serves_total',
                     'loser_first_serve_points_won','loser_first_serve_points_total','loser_second_serve_points_won',
                     'loser_second_serve_points_total','loser_break_points_saved','loser_break_points_serve_total',
                     'loser_service_points_won','loser_service_points_total','loser_first_serve_return_won',
                     'loser_first_serve_return_total','loser_second_serve_return_won','loser_second_serve_return_total',
                     'loser_break_points_converted','loser_break_points_return_total','loser_service_games_played',
                     'loser_return_games_played','loser_return_points_won','loser_return_points_total','loser_total_points_won',
                     'loser_total_points_total']
matchstats.columns = matchstatscolumns
# matchstats.head()

In [32]:
# Drop empty rows in matchstats.
nan_rows_match_time = matchstats[ (matchstats['match_time'].isnull() == True) ].index
matchstats.drop(nan_rows_match_time, inplace=True)

In [33]:
# Drop matches(rows) where there were either no total points, or no service points by both the winner and loser.
zero_points = matchstats[ (matchstats['winner_total_points_total'] == 0) ].index
matchstats.drop(zero_points, inplace=True)
zero_wservice_points = matchstats[ (matchstats['winner_service_points_total'] == 0) ].index
matchstats.drop(zero_wservice_points, inplace=True)
zero_lservice_points = matchstats[ (matchstats['loser_service_points_total'] == 0) ].index
matchstats.drop(zero_lservice_points, inplace=True)

In [34]:
# Drop matches(rows) where there were fewer than 12 total games played (incomplete match).
not_enough_winner_serve = matchstats[ (matchstats['winner_service_games_played'] < 6) ].index
matchstats.drop(not_enough_winner_serve, inplace=True)
not_enough_loser_serve = matchstats[ (matchstats['loser_service_games_played'] < 6) ].index
matchstats.drop(not_enough_loser_serve, inplace=True)

In [35]:
# Drop unnecessary columns in matchstats.
matchstats.drop(["tourney_order","match_stats_url_suffix","match_time"], inplace=True, axis=1)

In [36]:
matchstats.isnull().sum()

match_id                            0
match_duration                      0
winner_aces                         0
winner_double_faults                0
winner_first_serves_in              0
winner_first_serves_total           0
winner_first_serve_points_won       0
winner_first_serve_points_total     0
winner_second_serve_points_won      0
winner_second_serve_points_total    0
winner_break_points_saved           0
winner_break_points_serve_total     0
winner_service_points_won           0
winner_service_points_total         0
winner_first_serve_return_won       0
winner_first_serve_return_total     0
winner_second_serve_return_won      0
winner_second_serve_return_total    0
winner_break_points_converted       0
winner_break_points_return_total    0
winner_service_games_played         0
winner_return_games_played          0
winner_return_points_won            0
winner_return_points_total          0
winner_total_points_won             0
winner_total_points_total           0
loser_aces  

### Bring in rankings data for winners1 and losers1 dataframes into the matchstats dataframe.

In [37]:
matchstats1 = pd.merge(matchstats,winner1,on=["match_id"], how="inner")
matchstats1 = pd.merge(matchstats1,loser1,on=["match_id"], how="inner")

In [38]:
# Drop redundant and/or irrelevant columns in matchstats1.
matchstats1.drop(["tourney_year_id_x","match_score_tiebreaks_x","BeginWeek_x","winner_rank_text","tourney_year_id_y","loser_rank_text"], inplace=True, axis=1)

In [39]:
matchstats1.isnull().sum()

match_id                  0
match_duration            0
winner_aces               0
winner_double_faults      0
winner_first_serves_in    0
                         ..
loser_games_won           0
loser_tiebreaks_won       0
BeginWeek_y               0
loser_rank_number         0
loser_ranking_points      0
Length: 64, dtype: int64

### Create EDA Variables

In [40]:
matchstats1 = matchstats1.assign(winner_ace_pct =lambda matchstats1: (matchstats1['winner_aces'] / matchstats1['winner_service_points_total']) * 100)
matchstats1 = matchstats1.assign(loser_ace_pct =lambda matchstats1: (matchstats1['loser_aces'] / matchstats1['loser_service_points_total']) * 100)


In [41]:
matchstats1 = matchstats1.assign(winner_df_pct =lambda matchstats1: (matchstats1['winner_double_faults'] / matchstats1['winner_service_points_total']) * 100)
matchstats1 = matchstats1.assign(loser_df_pct =lambda matchstats1: (matchstats1['loser_double_faults'] / matchstats1['loser_service_points_total']) * 100)


In [42]:
matchstats1 = matchstats1.assign(winner_srv_pts_pct =lambda matchstats1: (matchstats1['winner_service_points_won'] / matchstats1['winner_service_points_total']) * 100)
matchstats1 = matchstats1.assign(loser_srv_pts_pct =lambda matchstats1: (matchstats1['loser_service_points_won'] / matchstats1['loser_service_points_total']) * 100)


In [43]:
matchstats1 = matchstats1.assign(winner_rtn_pts_pct =lambda matchstats1: (matchstats1['winner_return_points_won'] / matchstats1['winner_return_points_total']) * 100)
matchstats1 = matchstats1.assign(loser_rtn_pts_pct =lambda matchstats1: (matchstats1['loser_return_points_won'] / matchstats1['loser_return_points_total']) * 100)

In [44]:
matchstats1 = matchstats1.assign(winner_brk_pts_pct =lambda matchstats1: (matchstats1['winner_break_points_converted'] / matchstats1['winner_break_points_return_total']) * 100)
matchstats1 = matchstats1.assign(loser_brk_pts_pct =lambda matchstats1: (matchstats1['loser_break_points_converted'] / matchstats1['loser_break_points_return_total']) * 100)


In [45]:
matchstats1 = matchstats1.assign(winner_points_won_pct =lambda matchstats1: (matchstats1['winner_total_points_won'] / matchstats1['winner_total_points_total']) * 100)
matchstats1 = matchstats1.assign(loser_points_won_pct =lambda matchstats1: (matchstats1['loser_total_points_won'] / matchstats1['loser_total_points_total']) * 100)


In [46]:
matchstats1['loser_rtn_pts_pct'].fillna(0, inplace=True)
matchstats1['winner_brk_pts_pct'].fillna(0, inplace=True)
matchstats1['loser_brk_pts_pct'].fillna(0, inplace=True)

In [47]:
matchstats1 = matchstats1.assign(ace_dif =lambda matchstats1: (matchstats1['winner_ace_pct'] - matchstats1['loser_ace_pct']))
matchstats1 = matchstats1.assign(df_dif =lambda matchstats1: (matchstats1['winner_df_pct'] - matchstats1['loser_df_pct']))
matchstats1 = matchstats1.assign(srv_pts_dif =lambda matchstats1: (matchstats1['winner_srv_pts_pct'] - matchstats1['loser_srv_pts_pct']))
matchstats1 = matchstats1.assign(rtn_pts_dif =lambda matchstats1: (matchstats1['winner_rtn_pts_pct'] - matchstats1['loser_rtn_pts_pct']))
matchstats1 = matchstats1.assign(brk_pts_dif =lambda matchstats1: (matchstats1['winner_brk_pts_pct'] - matchstats1['loser_brk_pts_pct']))
matchstats1 = matchstats1.assign(rank_dif =lambda matchstats1: (matchstats1['winner_rank_number'] - matchstats1['loser_rank_number']))
matchstats1 = matchstats1.assign(total_points_pct_dif =lambda matchstats1: (matchstats1['winner_points_won_pct'] - matchstats1['loser_points_won_pct']))

In [48]:
matchstats1.isnull().sum()

match_id                   0
match_duration             0
winner_aces                0
winner_double_faults       0
winner_first_serves_in     0
                          ..
srv_pts_dif                0
rtn_pts_dif                0
brk_pts_dif                0
rank_dif                   0
total_points_pct_dif      89
Length: 83, dtype: int64

In [49]:
matchstats1.describe()

,match_duration,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,winner_first_serve_points_total,winner_second_serve_points_won,winner_second_serve_points_total,winner_break_points_saved,...,loser_brk_pts_pct,winner_points_won_pct,loser_points_won_pct,ace_dif,df_dif,srv_pts_dif,rtn_pts_dif,brk_pts_dif,rank_dif,total_points_pct_dif
count,88249.000000,88249.000000,88249.000000,88249.000000,88249.000000,88249.000000,88249.000000,88249.000000,88249.000000,88249.000000,...,88249.000000,88249.000000,88160.000000,88249.000000,88249.000000,88249.000000,88249.000000,88249.000000,88249.000000,88160.000000
mean,99.657084,6.237419,2.728167,45.308593,77.115265,35.211436,46.839704,16.605401,30.275561,3.368752,...,31.589918,55.468091,44.567113,2.742093,-1.040129,11.529555,11.518338,17.949813,-52.054040,10.893583
std,38.672842,5.092993,2.340265,20.234890,28.302553,13.179096,18.401658,6.800410,13.027984,3.106680,...,28.106883,4.115919,4.145333,7.339802,3.907797,9.119715,9.213576,33.897024,195.183544,8.215438
min,1.000000,0.000000,0.000000,0.000000,22.000000,3.000000,9.000000,0.000000,0.000000,0.000000,...,0.000000,30.303030,17.241379,-31.668052,-35.416667,-39.393939,-37.543860,-375.000000,-2126.000000,-25.274725
25%,73.000000,3.000000,1.000000,32.000000,56.000000,26.000000,33.000000,12.000000,21.000000,1.000000,...,0.000000,52.525253,42.287846,-1.708619,-3.340081,5.032468,5.004314,-1.136364,-75.000000,5.019305
50%,93.000000,5.000000,2.000000,43.000000,72.000000,33.000000,43.000000,16.000000,28.000000,3.000000,...,28.571429,54.819277,45.192308,2.086721,-0.891862,10.270270,10.260870,18.181818,-24.000000,9.615385
75%,121.000000,9.000000,4.000000,56.000000,93.000000,42.000000,57.000000,20.000000,37.000000,5.000000,...,50.000000,57.731959,47.490347,6.666667,1.458059,16.767984,16.758981,40.000000,19.000000,15.422886
max,1412.000000,113.000000,23.000000,361.000000,491.000000,292.000000,361.000000,82.000000,130.000000,24.000000,...,400.000000,82.758621,73.043478,55.002371,18.987342,68.292683,68.292683,800.000000,1777.000000,65.517241


In [50]:
# df2.loc[startrow:endrow, startcolumn:endcolumn]
matchstats1.iloc[:20,61:]


,BeginWeek_y,loser_rank_number,loser_ranking_points,winner_ace_pct,loser_ace_pct,winner_df_pct,loser_df_pct,winner_srv_pts_pct,loser_srv_pts_pct,winner_rtn_pts_pct,...,loser_brk_pts_pct,winner_points_won_pct,loser_points_won_pct,ace_dif,df_dif,srv_pts_dif,rtn_pts_dif,brk_pts_dif,rank_dif,total_points_pct_dif
0,1990-12-31,56.0,0.0,4.615385,1.449275,0.000000,0.000000,63.076923,53.623188,46.376812,...,100.000000,54.477612,45.522388,3.166109,0.000000,9.453735,9.453735,-42.857143,-5.0,8.955224
1,1990-12-31,42.0,0.0,1.538462,14.457831,3.076923,6.024096,66.153846,55.421687,44.578313,...,60.000000,54.054054,45.945946,-12.919370,-2.947173,10.732159,10.732159,-2.857143,9.0,8.108108
2,1990-12-31,62.0,0.0,0.000000,3.703704,2.777778,3.703704,66.666667,37.037037,62.962963,...,66.666667,64.444444,35.555556,-3.703704,-0.925926,29.629630,29.629630,3.333333,-11.0,28.888889
3,1990-12-31,108.0,0.0,3.448276,5.357143,5.172414,7.142857,60.344828,51.785714,48.214286,...,25.000000,54.385965,45.614035,-1.908867,-1.970443,8.559113,8.559113,30.555556,-48.0,8.771930
4,1990-12-31,50.0,0.0,8.450704,6.250000,4.225352,3.125000,61.971831,50.000000,50.000000,...,66.666667,56.296296,43.703704,2.200704,1.100352,11.971831,11.971831,-16.666667,32.0,12.592593
5,1990-12-31,304.0,0.0,11.494253,14.285714,0.000000,1.098901,74.712644,73.626374,26.373626,...,0.000000,50.000000,50.000000,-2.791461,-1.098901,1.086270,1.086270,0.000000,-248.0,0.000000
6,1990-12-31,25.0,0.0,8.750000,2.816901,5.000000,2.816901,66.250000,66.197183,33.802817,...,16.666667,50.993377,49.006623,5.933099,2.183099,0.052817,0.052817,50.000000,17.0,1.986755
7,1990-12-31,60.0,0.0,6.756757,2.469136,2.702703,2.469136,70.270270,62.962963,37.037037,...,50.000000,52.903226,47.096774,4.287621,0.233567,7.307307,7.307307,-7.142857,-18.0,5.806452
8,1990-12-31,66.0,0.0,9.523810,3.125000,6.349206,3.125000,66.666667,51.562500,48.437500,...,20.000000,57.480315,42.519685,6.398810,3.224206,15.104167,15.104167,46.666667,-24.0,14.960630
9,1990-12-31,54.0,0.0,3.636364,3.125000,0.000000,4.687500,69.090909,54.687500,45.312500,...,50.000000,56.302521,43.697479,0.511364,-4.687500,14.403409,14.403409,-10.000000,-29.0,12.605042


### Export Completed File to csv format for use in EDA

In [51]:
matchstats1.to_csv(r'data/matchstats.csv')